In [1]:
# Libraries to be used 
import pandas as pd
import numpy as np
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
%matplotlib inline
sns.set(color_codes=True)

In [2]:
# Import the file into a dataframe 

try:
    aps_failure_set=pd.read_csv(r"\\sedna\jokin.ormazabal$\$Profile\Desktop\Master\Data Prep\aps_failure_set.csv")
    print("APS Failure Set successfully imported into a dataframe")
except:
    print("Something went wrong importing the APS Failure Set")

#As comma is the defaulted delimiter, we dont need to specify it

#try to do it trough a url



APS Failure Set successfully imported into a dataframe


######  Based on the explanation received, we know the data divides between the errors relating APS and not, 
so we divide the main Dataframe into 2 different ones.

In [3]:
aps_failures=aps_failure_set.loc[aps_failure_set['class']=='pos']
not_aps_failures=aps_failure_set.loc[aps_failure_set['class']=='neg']

###### EXPLANATORY DATA ANALYSIS FOR APS RELATED FAILURES

In [4]:

na_values_aps_failures = aps_failures.isna().sum()
na_aps = na_values_aps_failures.to_numpy()

if np.sum(na_aps) == 0:
    print('The APS failures dataset has NO na values')
else:
    print('The APS failures dataset has na values')


    
    

The APS failures dataset has NO na values


######  We know by looking at the data that it actually has na values but the function is not catching them. 
This happens because the na is a string and for python the value is actually not null. 
It is important to always manually review the data and check that the output from the code make sense.

In [5]:
# We will now look for values equal to 'na'

na_values_aps_failures=aps_failures.isin(['na']) #creates a df with True for values=='na' & false for values !='na'

if (na_values_aps_failures.any().any())==True: 
    print('The APS failures dataset has values equal to na')
else:
     print('The APS failures dataset has NO values equal to na')

The APS failures dataset has values equal to na


######  As we suspected, we actually have null values. 
These values will need to be excluded from any statistical analysis and python has ways to do so. 
The best method will be to convert those na values into propper NaN's so that the code recognises them latter.

In [6]:
aps_failures=aps_failures.replace("na",np.nan) 
aps_failures=aps_failures.replace("",np.nan) # In case there are any blank values, we convert them as NaN as well

na_aps = na_values_aps_failures.to_numpy()

if np.sum(na_aps) == 0:
    print('The APS failures dataset has NO na values')
else:
    print('The APS failures dataset has na values')



The APS failures dataset has na values


######  Now that we identified the existance of NaN values we will contextualize them and decide if we can drop them. 

In [7]:
aps_failures.shape 

(1000, 171)

######  We know that we have 1000 rows and 171 columns. So now It will interesting to see which columns have a great % of nulls 

In [8]:
null_percentages=(aps_failures.isnull().sum()/1000*100)
null_percentages = aps_failures.isnull().mean() * 100 # we can also do it with the mean attribute

In [9]:
High_null_aps_failures = pd.DataFrame(null_percentages, columns=['High_Null']) #Data frame with the % of null values by column
print (High_null_aps_failures)

        High_Null
class         0.0
aa_000        0.0
ab_000       77.1
ac_000       46.2
ad_000       64.5
...           ...
ee_007        0.5
ee_008        0.5
ee_009        0.5
ef_000       37.7
eg_000       37.7

[171 rows x 1 columns]


In [10]:
High_null_aps_failures.reset_index(drop=False, inplace=True) #I need to add an index as the sensor names where not counting as a propper column

In [11]:
# print (High_null_aps_failures)

In [12]:
High_null_aps_failures=High_null_aps_failures[High_null_aps_failures.iloc[:,1]>20] #Select the sensors with greater than 20% null values

High_null_aps_failures.shape

(60, 2)

######  We see that there are 60 sensors with more than 20% of null values. I consider that number too high to analyse them

###### I'll drop all those columns to focus only on the most reliable data.

In [13]:
faulty_sensors=High_null_aps_failures.iloc[:,0].tolist() #get the name of the faulty sensors on a list 

In [14]:
aps_failures=aps_failures.drop(faulty_sensors,axis=1)
aps_failures.shape

(1000, 111)

######  We droped what I considered faulty sensors by the high null values, but still we have nulls on the other sensors.
###### The existance of this nulls makes the sample incosistant as we have more records for some sensors. 
###### We will drop the rows with null values to make the data set equal for every sensor type.


In [15]:
aps_failures = aps_failures.dropna()
aps_failures.shape

(861, 111)

###### We have cleaned our dataset from nulls and "faulty" data, but still we have a high number of different sensors to analyse

###### As each number represents the Component Sensor result we cant group the data with the mean. (I assume the number is the type of error, not the number of errors)

###### I'll use the mode to detect the most repeated error type by sensor

In [16]:
aps_failures.set_index(aps_failures.columns[0],inplace=True) # To calculate the mode I need to exclude the first column, so I'll make it the index

aps_failures=aps_failures.astype(float) # Convert all the df to float type. It got converted to int with so many cleaning

In [17]:
df_melted = aps_failures.melt(var_name='Sensor', value_name='Most Common Error') # Group the number of errors by Sensor type

In [18]:
print (df_melted)

       Sensor  Most Common Error
0      aa_000           453236.0
1      aa_000          1056758.0
2      aa_000           361638.0
3      aa_000           791254.0
4      aa_000          1053152.0
...       ...                ...
94705  ee_009            10790.0
94706  ee_009             2458.0
94707  ee_009                0.0
94708  ee_009                0.0
94709  ee_009                0.0

[94710 rows x 2 columns]


In [19]:
# Get the mode of each column
mode = aps_failures.mode().iloc[0]

# Convert to DataFrame and reset index
df_aps_modes = pd.DataFrame(mode).reset_index()

# Rename the columns
df_aps_modes.columns = ['Sensor', 'Most Common Error']

df_aps_modes

,Sensor,Most Common Error
0,aa_000,181460.0
1,ag_000,0.0
2,ag_001,0.0
3,ag_002,0.0
4,ag_003,0.0
...,...,...
105,ee_005,0.0
106,ee_006,0.0
107,ee_007,0.0
108,ee_008,0.0


###### Select only sensors with an error. (I assume that the 0 means no error)

In [20]:
df_aps_modes=df_aps_modes[df_aps_modes['Most Common Error'] != 0]

print (df_aps_modes)

    Sensor  Most Common Error
0   aa_000          181460.00
11  ah_000        14818484.00
16  an_000        16230962.00
17  ao_000        13692914.00
18  ap_000         6904476.00
19  aq_000         3618134.00
53  bb_000        24654966.00
54  bg_000        14818484.00
55  bh_000          363444.00
56  bi_000         1550620.00
57  bj_000         5339498.00
58  bk_000          387860.00
59  bl_000          393520.00
60  bm_000          424600.00
61  bn_000          438120.00
62  bo_000         1310700.00
63  bp_000         1310700.00
64  bq_000         1310700.00
65  br_000         1310700.00
66  bs_000           22880.00
67  bt_000          181459.46
68  bu_000        24654966.00
69  bv_000        24654966.00
70  bx_000             226.00
72  cb_000           42860.00
74  cd_000         1209600.00
75  ci_000        14244931.20
77  ck_000        10022479.68
88  cq_000        24654966.00
99  dn_000          242252.00


In [21]:
df_aps_modes.shape

(30, 2)

###### We get the mode again to see which is the most common failure id for the APS related issues

In [22]:
mode = df_aps_modes['Most Common Error'].mode()
print(mode)

0     1310700.0
1    24654966.0
Name: Most Common Error, dtype: float64


######  At this stage, we can derive some conclusions for the APS related failures. 
    - 60 out of 171 were considered failed sensors because 20% or more than their results were null (35%)
    - 139 records out 1000 needed to be excluded as one of the sensors triggered null value (14%)
    - 30 from the 111 sensors that were took into the analysis tiggered some kind of error (27%)
    - 1310700 & 24654966 are the most common failures related with the APS

##### EXPLANATORY DATA ANALYSIS FOR NOT APS RELATED FAILURES


###### We replace na for nan so the code can detect them as we learned before.

In [23]:

#not_aps_failures=not_aps_failures.drop(['class'],axis=1)

not_aps_failures=not_aps_failures.replace("na",np.nan) 
not_aps_failures=not_aps_failures.replace("",np.nan) 

if not_aps_failures.isnull().values.any():
    print('The Not APS failures dataset has na values')
else:
    print('The Not APS failures dataset has NO na values')


The Not APS failures dataset has na values


In [24]:
not_aps_failures=not_aps_failures.drop(['class'],axis=1)

In [25]:
aps_failures=aps_failures.replace("na",np.nan) 
aps_failures=aps_failures.replace("",np.nan) # In case there are any blank values, we convert them as NaN as well

na_aps = na_values_aps_failures.to_numpy()

if np.sum(na_aps) == 0:
    print('The APS failures dataset has NO na values')
else:
    print('The APS failures dataset has na values')


The APS failures dataset has na values


Index(['aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000',
       'ag_001', 'ag_002', 'ag_003',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=170)
